In [1]:
# jax
import jax
# jax.config.update("jax_default_device",jax.devices()[1])
jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp
from jax import jit,grad,jacfwd,jacrev,vmap
from jax.random import PRNGKey as pkey
from jax.scipy.linalg import solve
# Other libraries
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
plt.style.use("ggplot")
from importlib import reload
from functools import partial

# Our libraries
import KernelTools
reload(KernelTools)
from KernelTools import *
from EquationModel import (OperatorModel, OperatorPDEModel,CholInducedRKHS,
                            CholOperatorModel)
import plotting
reload(plotting)
from plotting import (plot_obs,plot_compare_error,
                      plot_input_data,plot_compare_error,
                      plot_obs_parabolic, plot_init_final_parabolic)

from evaluation_metrics import get_nrmse,table_u_errors
from data_utils import (
    get_xy_grid_pairs, # changed - could live in parabolic_data_utils ?
    build_xy_grid,
    sample_xy_grid_latin,
    build_input_observations,
    build_u_obs_single,
    build_u_obs_all,
    sample_multiple_xy_grids_latin
)

from parabolic_data_utils import (
    build_burgers_data,build_tx_grid,
    build_tx_grid_chebyshev,setup_problem_data,
    GP_Sampler_1D_Pinned
)

from Kernels import (
    get_gaussianRBF,
    get_centered_scaled_poly_kernel,
    fit_kernel_params,log1pexp,
    inv_log1pexp,get_anisotropic_gaussianRBF
)

In [2]:
def run_exp_opt_err(alpha,n_obs,run):

    # Sample m training functions as initial conditions from a GP
    m = 1
    kernel_GP = get_gaussianRBF(0.2)
    xy_pts = jnp.linspace(0,1,50) # Pts to build interpolants
    u0_true_functions = tuple(GP_Sampler_1D_Pinned(num_samples = m,
                                            X = xy_pts, 
                                            smooth = alpha,
                                            kernel = kernel_GP,
                                            reg = 1e-8,
                                            seed = 2024
                                        )
                            )

    vec_u0_true_functions = tuple([np.vectorize(u) for u in u0_true_functions])
    vmapped_u0_true_functions = tuple([jax.vmap(u) for u in u0_true_functions])
    # Solve m Burgers' equations with IC's: u0_true_functions 
    # kappa = 0.001
    # alpha = 5.
    kappa = 0.01
    alpha = 1.

    u_true_functions = []
    ut_true_functions = []
    interpolants = []
    t_vals_s = []
    sols_s = []

    for u0_true_function in vec_u0_true_functions:
        res = list(build_burgers_data(func_u0=u0_true_function,
                        kappa = kappa,
                        alpha = alpha,
                        k_timestep=0.0001,
                        n_finite_diff=1999))
    u_true_functions.append(res[0])
    ut_true_functions.append(res[1])
    interpolants.append(res[2])
    t_vals_s.append(res[3])
    sols_s.append(res[4])

    # Ghost Chebyshev grids
    num_grid_x = 31
    num_grid_t = 26
    # num_grid_x = 7
    # num_grid_t = 6

    tx_ints , tx_bdys = [], []
    for _ in range(m):
        res = build_tx_grid_chebyshev([0,1],[0,1],
                                num_grid_t,
                                num_grid_x,
                                alpha = 0.5)
        tx_ints.append(res[0])
        tx_bdys.append(res[1])

    tx_ints = tuple(tx_ints)
    tx_bdys = tuple(tx_bdys)
    # Tuple of ghost grids
    tx_all = tuple(jnp.vstack([tx_int,tx_bdy]) for tx_int,tx_bdy in zip(tx_ints,tx_bdys))


    # List of number of observation points per u
    num_obs = [n_obs]*m

    # Get (X^m, u^m(X^m))
    tx_obs = []
    u_obs = []
    for i, (tx_int,tx_bdy) in enumerate(zip(tx_ints,tx_bdys)):
        _,tx_ob = setup_problem_data(
            tx_int,
            tx_bdy,
            num_obs[i],
            jax.random.PRNGKey(i),
            times_to_observe = (0,)
                )
        tx_obs.append(tx_ob)

    u_obs = tuple(u(tx_ob) for tx_ob,u in zip(tx_obs,u_true_functions))

    # Define grid to plot
    num_fine_grid = 300
    initial_time = 0
    final_time = 1
    t_fine,x_fine = jnp.meshgrid(jnp.linspace(initial_time,final_time,num_fine_grid),jnp.linspace(0,1,num_fine_grid))
    tx_fine = jnp.vstack([t_fine.flatten(),x_fine.flatten()]).T

    # Plot initial and final conditions
    num_fine_grid = 500
    grid_points = jnp.linspace(0,1,num_fine_grid)

    # Features of u and P
    u_operators = (eval_k,) # This enforces 1_5 step method
    feature_operators = (eval_k,dx_k,dxx_k)


    # Fit parameters of kernel for u
    def param_ani_gaussian_RBF(x,y,params):
        lengthscales = log1pexp(params)
        return get_anisotropic_gaussianRBF(1.,jnp.diag(lengthscales))(x,y)

    fitted_params_s = []
    ml_value_s = []
    k_u_s = []
    for i in range(m):
        res = fit_kernel_params(param_ani_gaussian_RBF,tx_obs[i],u_obs[i],jnp.zeros(2))
        ML_lengthscales = log1pexp(res[0])
        k_u_s.append(get_anisotropic_gaussianRBF(1.,jnp.diag(log1pexp(res[0]))))
        #print(1/(jnp.sqrt(ML_lengthscales)))

    u_models = tuple([CholInducedRKHS(
        tx_all[i],
        u_operators,
        k_u_s[i]
        ) for i in range(m)])
    

    # Get necessary tuples
    observation_points = tuple(tx_obs)
    observation_values = tuple(u_obs)
    collocation_points = tx_ints

    all_u_params_init = tuple([
        model.get_fitted_params(obs_loc,obs_val)
        for obs_loc,obs_val,model in zip(observation_points,observation_values,u_models)])

    grid_features_u_init = jnp.vstack([(
        model.evaluate_operators(feature_operators,tx_int,model_params)).reshape(
                len(tx_int),
                len(feature_operators),
                order = 'F'
            ) for tx_int,model,model_params in zip(tx_ints,u_models,all_u_params_init)])

    k_P_u_part = get_centered_scaled_poly_kernel(2,grid_features_u_init,c=1.,scaling = 'diagonal')

    @vectorize_kfunc
    def k_P(x,y):
        return k_P_u_part(x[2:],y[2:])

    P_model = OperatorModel(k_P)

    EqnModel  = OperatorPDEModel(
        P_model,
        u_models,
        observation_points,
        observation_values,
        collocation_points,
        feature_operators,
        rhs_operator=dt_k,
        datafit_weight = 5.,
        )

    import LM_Solve
    import importlib
    importlib.reload(LM_Solve)
    from LM_Solve import LevenbergMarquadtMinimize,adaptive_refine_solution
    params_init = jnp.hstack(list(all_u_params_init)+[jnp.zeros(m*len(tx_ints[0]))])

    params,convergence_data = LevenbergMarquadtMinimize(
        params_init.copy(),
        EqnModel,
        beta = 1e-11,
        max_iter = 501,
        init_alpha=0.1,
        line_search_increase_ratio=1.4,
        print_every = 100
    )
    p_adjusted,refine_convergence_data = adaptive_refine_solution(params,EqnModel,1e-3,500)

    equation_residuals = EqnModel.equation_residual(params)
    equation_residuals_refined = EqnModel.equation_residual(p_adjusted)

    u_sols = EqnModel.get_u_params(p_adjusted)
    P_sol = EqnModel.get_P_params(p_adjusted)

    u_sols = EqnModel.get_u_params(params)
    P_sol = EqnModel.get_P_params(params)



    #### Operator learning error : 1.5 step method
    # Sample m u0 new functions from a GP
    M = 15
    kernel_GP = get_gaussianRBF(0.2)
    xy_pts = jnp.linspace(0,1,50) # Pts to build interpolants
    u0_new_true_functions = tuple(GP_Sampler_1D_Pinned(num_samples = M,
                                            X = xy_pts, 
                                            smooth= alpha,
                                            kernel = kernel_GP,
                                            reg = 1e-8,
                                            seed = run
                                        )
                            )

    vec_u0_new_true_functions = tuple([np.vectorize(u) for u in u0_new_true_functions])
    vmapped_u0_new_true_functions = tuple([jax.vmap(u) for u in u0_new_true_functions])
    # Solve m Burgers' equations with IC's: u0_true_functions 
    # kappa = 0.001
    # alpha = 5.
    kappa = 0.01
    alpha = 1.

    u_new_true_functions = []
    ut_new_true_functions = []
    interpolants_new = []
    t_vals_s_new = []
    sols_s_new = []

    for u0_new_true_function in vec_u0_new_true_functions:
        res = list(build_burgers_data(func_u0=u0_new_true_function,
                        kappa = kappa,
                        alpha = alpha,
                        k_timestep=0.0001,
                        n_finite_diff=1999))
        u_new_true_functions.append(res[0])
        ut_new_true_functions.append(res[1])
        interpolants_new.append(res[2])
        t_vals_s_new.append(res[3])
        sols_s_new.append(res[4])

    # Colocation points for learning u
    num_grid_x = 31
    num_grid_t = 26
    tx_int,tx_bdy = build_tx_grid_chebyshev([0,1],[0,1],num_grid_t,num_grid_x,alpha = 0.5)

    tx_all,_ = setup_problem_data(
        tx_int,
        tx_bdy,
        60,
        jax.random.PRNGKey(55),
        times_to_observe = (0,)
    )

    new_u_eval_fines = []
    for i in range(M):
        model_grid_features = EqnModel.get_stacked_eqn_features(u_sols)
        @jit
        def get_pred_ut(input_features):
            return P_model.kernel_function(input_features,model_grid_features)@P_sol

        tx_init_bdy = jnp.vstack([tx_int[tx_int[:,0]==0],tx_bdy])

        new_obs = u_new_true_functions[i](tx_init_bdy)    

        tx_all_ = tx_all[0]

        # Object to capture learned solution : Fixing the first ?
        u_model = CholInducedRKHS(tx_all,
                                    feature_operators,
                                    k_u_s[0]
                                    )

        class PdeModel():
            datafit_weight = 50.
            @jit
            def get_grid_features(u_params):
                evaluation = u_model.evaluate_operators(feature_operators,tx_int,u_params)
                grid_features = evaluation.reshape(len(tx_int),len(feature_operators),order = 'F')
                return jnp.hstack([tx_int,grid_features])
            
            @jit
            def get_grid_target(u_params):
                return u_model.evaluate_operators((dt_k,),tx_int,u_params)
            
            @jit
            def eval_obs_points(u_params):
                return u_model.point_evaluate(tx_init_bdy,u_params)
            
            @jit
            def datafit_residual(u_params):
                obs_preds = PdeModel.eval_obs_points(u_params)
                return new_obs - obs_preds
            
            @jit
            def equation_residual(u_params):
                P_features = PdeModel.get_grid_features(u_params)
                P_model_preds = get_pred_ut(P_features)
                ugrid_target = PdeModel.get_grid_target(u_params)
                return (ugrid_target - P_model_preds)
            
            @jit
            def F(u_params):
                eqn_res = PdeModel.equation_residual(u_params)
                data_res = PdeModel.datafit_residual(u_params)
                return jnp.hstack([
                    PdeModel.datafit_weight * data_res/jnp.sqrt(len(data_res)),
                    eqn_res/jnp.sqrt(len(eqn_res))
                    ])
            
            jac = jit(jacrev(F))

            def loss(u_params):
                return jnp.linalg.norm(PdeModel.F(u_params))**2
            
            @jit
            def damping_matrix(u_params):
                return u_model.kmat+1e-5 * diagpart(u_model.kmat)
            

        # OPTIMIZE
        new_u_init = jnp.zeros(tx_all.shape[0]*len(feature_operators))

        u_params_pde,pde_convergence_data = LevenbergMarquadtMinimize(
            new_u_init.copy(),
            PdeModel,
            beta = 0.,
            max_iter = 101,
            init_alpha=0.05,
            line_search_increase_ratio=1.4,
            print_every = 10
        )
        u_pde_adj,pde_refine_conv_data = adaptive_refine_solution(u_params_pde,PdeModel,1e-1,150)

        new_u_eval_fines.append(u_model.point_evaluate(tx_fine,u_pde_adj))

    # True
    new_u_true_eval_fines = [u(tx_fine) for u in u_new_true_functions]
    # Pred
    new_u_eval_fines = new_u_eval_fines

    i_opt_1_5 = jnp.mean(jnp.array([get_nrmse(t,p) for t, p in zip(new_u_eval_fines,new_u_true_eval_fines)]))


    #### Operator learning error : 2 step method

    new_u_eval_fines_2step = []
    for i in range(M):
        model_grid_features = EqnModel.get_stacked_eqn_features(u_sols)

        tx_init_bdy = jnp.vstack([tx_int[tx_int[:,0]==0],tx_bdy])

        new_obs = u_new_true_functions[i](tx_init_bdy)    

        tx_all_ = tx_all[0]

        # Object to capture learned solution : Fixing the first ?
        u_model = CholInducedRKHS(tx_all,
                                    feature_operators,
                                    k_u_s[0]
                                    )

        @jit
        def get_pred_ut_2step(input_features):
            init_P_features = EqnModel.get_stacked_eqn_features(all_u_params_init)
            rhs_stacked = EqnModel.stacked_collocation_rhs
            P_params_naive = P_model.fit_params(init_P_features,rhs_stacked,nugget = 1e-8)
            return P_model.predict_new(input_features,init_P_features,P_params_naive)
        

        class PdeModel():
            datafit_weight = 50.
            @jit
            def get_grid_features(u_params):
                evaluation = u_model.evaluate_operators(feature_operators,tx_int,u_params)
                grid_features = evaluation.reshape(len(tx_int),len(feature_operators),order = 'F')
                return jnp.hstack([tx_int,grid_features])
            
            @jit
            def get_grid_target(u_params):
                return u_model.evaluate_operators((dt_k,),tx_int,u_params)
            
            @jit
            def eval_obs_points(u_params):
                return u_model.point_evaluate(tx_init_bdy,u_params)
            
            @jit
            def datafit_residual(u_params):
                obs_preds = PdeModel.eval_obs_points(u_params)
                return new_obs - obs_preds
            
            @jit
            def equation_residual(u_params):
                P_features = PdeModel.get_grid_features(u_params)
                P_model_preds = get_pred_ut_2step(P_features)
                ugrid_target = PdeModel.get_grid_target(u_params)
                return (ugrid_target - P_model_preds)
            
            @jit
            def F(u_params):
                eqn_res = PdeModel.equation_residual(u_params)
                data_res = PdeModel.datafit_residual(u_params)
                return jnp.hstack([
                    PdeModel.datafit_weight * data_res/jnp.sqrt(len(data_res)),
                    eqn_res/jnp.sqrt(len(eqn_res))
                    ])
            
            jac = jit(jacrev(F))

            def loss(u_params):
                return jnp.linalg.norm(PdeModel.F(u_params))**2
            
            @jit
            def damping_matrix(u_params):
                return u_model.kmat+1e-5 * diagpart(u_model.kmat)
            

        # OPTIMIZE
        new_u_init = jnp.zeros(tx_all.shape[0]*len(feature_operators))

        u_params_pde,pde_convergence_data = LevenbergMarquadtMinimize(
            new_u_init.copy(),
            PdeModel,
            beta = 0.,
            max_iter = 101,
            init_alpha=0.05,
            line_search_increase_ratio=1.4,
            print_every = 10
        )
        u_pde_adj,pde_refine_conv_data = adaptive_refine_solution(u_params_pde,PdeModel,1e-1,150)

        new_u_eval_fines_2step.append(u_model.point_evaluate(tx_fine,u_pde_adj))
    
    # True
    new_u_true_eval_fines = [u(tx_fine) for u in u_new_true_functions]
    # Pred
    new_u_eval_fines2step = new_u_eval_fines_2step

    i_opt_2 = jnp.mean(jnp.array([get_nrmse(t,p) for t, p in zip(new_u_eval_fines_2step,new_u_true_eval_fines)]))

    return i_opt_1_5, i_opt_2

In [3]:
err = {
    '1_5_mthd': [],
    '2_mthd':   []
}

In [4]:
NUM_RUNS = 3
REGULARITY = [1,2,3,4]
for alpha in REGULARITY:
    i_opt_1_5 = []
    i_opt_2 = []
    for run in range(NUM_RUNS):
        # Run
        res = run_exp_opt_err(alpha,60,run)
        # Append
        i_opt_1_5.append(res[0])
        i_opt_2.append(res[1])
    # Append each list    
    err['1_5_mthd'].append(i_opt_1_5)
    err['2_mthd'].append(i_opt_2)   
# Save after
jnp.save('errors_opt/err_vs_smooth', err)

  0%|          | 0/10099 [00:00<?, ?it/s]

  0%|          | 0/501 [00:00<?, ?it/s]

Iteration 0, loss = 0.4439, Jres = 44.89, alpha = 0.08333, improvement_ratio = 0.9712
Iteration 1, loss = 0.2334, Jres = 7.763, alpha = 0.06944, improvement_ratio = 1.006
Iteration 2, loss = 0.1549, Jres = 2.457, alpha = 0.05787, improvement_ratio = 1.018
Iteration 3, loss = 0.115, Jres = 1.103, alpha = 0.04823, improvement_ratio = 1.031
Iteration 4, loss = 0.09098, Jres = 0.7302, alpha = 0.04019, improvement_ratio = 1.053
Iteration 5, loss = 0.07271, Jres = 0.6387, alpha = 0.03349, improvement_ratio = 1.072
Iteration 100, loss = 0.006384, Jres = 0.000123, alpha = 8.333e-07, improvement_ratio = 1.0
Iteration 200, loss = 0.006171, Jres = 9.157e-06, alpha = 8.333e-07, improvement_ratio = 1.0
Iteration 300, loss = 0.00606, Jres = 5.466e-06, alpha = 8.333e-07, improvement_ratio = 1.0
Iteration 400, loss = 0.005981, Jres = 5.766e-06, alpha = 8.333e-07, improvement_ratio = 1.0
Iteration 500, loss = 0.005919, Jres = 5.734e-06, alpha = 8.333e-07, improvement_ratio = 1.0


  0%|          | 0/500 [00:00<?, ?it/s]

Iteration 0, loss = 0.005917640246862752
Iteration 93 Step Failed
Iteration 234 Step Failed
Iteration 306 Step Failed
Iteration 318 Step Failed
Iteration 360 Step Failed
Iteration 385 Step Failed
Iteration 412 Step Failed
Iteration 422 Step Failed
Iteration 423 Step Failed
Iteration 443 Step Failed
Iteration 452 Step Failed
Iteration 456 Step Failed
Iteration 462 Step Failed
Iteration 475 Step Failed
Iteration 494 Step Failed
Iteration 496 Step Failed
Iteration 497 Step Failed
Iteration 498 Step Failed
Iteration 499 Step Failed


...

Iteration 0, loss = 0.6425, Jres = 139.2, alpha = 0.04167, improvement_ratio = 0.9923
Iteration 1, loss = 0.1982, Jres = 1.887, alpha = 0.03472, improvement_ratio = 0.9393
Iteration 2, loss = 0.16, Jres = 0.2478, alpha = 0.02894, improvement_ratio = 1.003
Iteration 3, loss = 0.1548, Jres = 0.01862, alpha = 0.02411, improvement_ratio = 0.9781
Iteration 4, loss = 0.1525, Jres = 0.008538, alpha = 0.02009, improvement_ratio = 1.006
Iteration 5, loss = 0.1509, Jres = 0.003134, alpha = 0.01674, improvement_ratio = 1.002
Iteration 10, loss = 0.1449, Jres = 0.001647, alpha = 0.006729, improvement_ratio = 1.002
Iteration 20, loss = 0.1324, Jres = 0.001462, alpha = 0.001087, improvement_ratio = 1.001
Iteration 30, loss = 0.1247, Jres = 0.0008727, alpha = 0.0001755, improvement_ratio = 1.002
Iteration 40, loss = 0.1207, Jres = 0.0006683, alpha = 2.835e-05, improvement_ratio = 1.002
Iteration 50, loss = 0.117, Jres = 0.0005588, alpha = 4.579e-06, improvement_ratio = 1.002
Iteration 60, loss = 0.11

  0%|          | 0/150 [00:00<?, ?it/s]

Iteration 0, loss = 0.2087986324672432
Iteration 0 Step Failed
Iteration 1 Step Failed
Iteration 3 Step Failed
Iteration 4 Step Failed
Iteration 8 Step Failed
Iteration 9 Step Failed
Iteration 10 Step Failed
Iteration 11 Step Failed
Iteration 12 Step Failed
Iteration 14 Step Failed
Iteration 15 Step Failed
Iteration 16 Step Failed
Iteration 17 Step Failed
Iteration 18 Step Failed
Iteration 19 Step Failed
Iteration 20 Step Failed
Iteration 21 Step Failed
Iteration 22 Step Failed
Iteration 23 Step Failed
Iteration 24 Step Failed
Iteration 25 Step Failed
Converged by no improvement


2024-09-27 09:37:23.526881: W external/xla/xla/tsl/framework/bfc_allocator.cc:482] Allocator (GPU_0_bfc) ran out of memory trying to allocate 8.65GiB (rounded to 9285121024)requested by op 
2024-09-27 09:37:23.528051: W external/xla/xla/tsl/framework/bfc_allocator.cc:494] ****************************************************________________________________________________
E0927 09:37:23.528070 2835149 pjrt_stream_executor_client.cc:2985] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 9285120928 bytes.


XlaRuntimeError: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 9285120928 bytes.